In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F

In [4]:
lstm = nn.LSTM(1, 6,batch_first = True)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

In [5]:
for i in inputs:
    print i.size()
    print i.view(1,1,-1).size()
    out, hidden = lstm(i.view(1, -1, 1))
    break

torch.Size([1, 3])
torch.Size([1, 1, 3])


In [6]:
out.size()

torch.Size([1, 3, 6])

In [7]:
out[:,-1,:].size()

torch.Size([1, 6])

In [8]:
out.view(3, -1).size()

torch.Size([3, 6])

-------------

In [10]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [11]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book apple".split(), ["NN", "V", "DET", "NN", 'NN'])
]

In [12]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

{'Everybody': 5, 'ate': 2, 'apple': 4, 'that': 7, 'read': 6, 'dog': 1, 'book': 8, 'the': 3, 'The': 0}


In [13]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [14]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim,batch_first = True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #print embeds.size()
        lstm_out, self.hidden = self.lstm(embeds)
        tag_space = self.hidden2tag(lstm_out)
        #tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_space


In [15]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [18]:
for epoch in range(1):  # again, normally you would NOT do 300 epochs, it is toy data
    sent = []
    tar = []
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        sentence_in = sentence_in.view(1,-1)
        targets = targets.view(1,-1)
        sent.append(sentence_in)
        tar.append(targets)
       # print sentence_in.size()
        #print targets.size()
        #print targets
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        #print tag_scores.view(-1,tag_scores.size(2)).size()
        #print "===="
        #print  targets.view(-1).size()
        #print tag_scores.view(-1,tag_scores.size(2)).size()
        #print targets
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores.view(-1,tag_scores.size(2)), targets.view(-1))
        print loss
        loss.backward()
        optimizer.step()

tensor(1.1549)
tensor(1.1021)


In [19]:
sent

[tensor([[ 0,  1,  2,  3,  4]]), tensor([[ 5,  6,  7,  8,  4]])]

In [20]:
torch.cat(tar,dim=0)

tensor([[ 0,  1,  2,  0,  1],
        [ 1,  2,  0,  1,  1]])

In [21]:
torch.cat(sent,dim=0)

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  4]])

In [22]:
tag_scores = model(torch.cat(sent,dim=0))

In [23]:
tag_scores.size()

torch.Size([2, 5, 3])

In [24]:
tag_scores.view(-1,tag_scores.size(2)).size()

torch.Size([10, 3])

In [26]:
torch.cat(tar,dim=0).view(-1).size()

torch.Size([10])

In [194]:
loss = loss_function(tag_scores.view(-1,tag_scores.size(2)), torch.cat(tar,dim=0).view(-1))
print loss
loss.backward()
optimizer.step()

tensor(1.0977)


-------------------------